In [1]:
!pip install torch_geometric

In [1]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-sparse 0.6.0
    Uninstalling torch-sparse-0.6.0:
      Successfully uninstalled torch-sparse-0.6.0


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import time
import torch.nn.functional as F

In [0]:
from utils import EarlyStopping

In [0]:
from torch_geometric.datasets import Planetoid

In [0]:
Cora = Planetoid(root='/content/drive/My Drive/', name='Cora')
cora = Cora[0]

In [0]:
C_S = Planetoid(root='/content/drive/My Drive/', name='CiteSeer')
cite_seer = C_S[0]

In [0]:
PM = Planetoid(root='/content/drive/My Drive/', name='PubMed')
pub_med = PM[0]

In [0]:
#cora = torch.load('Con_Cora.pt')

In [0]:
#cite_seer = torch.load('Con_Cite.pt')


In [0]:
def Gen_data(dataset, num_features,seed):
    np.random.seed(seed)
    data = dataset.clone()
    rx = np.random.normal(0, 1, size=(data.num_nodes, num_features) )
    #rx = np.random.random((data.num_nodes, num_features))
    rx = torch.FloatTensor(rx)
    data.x = rx
    return data

In [0]:
# Cora
cora_seed = 1024
N_F = 100
r_cora = Gen_data(cora, N_F, cora_seed)

In [0]:
#Cite_Seer
cite_seer_seed = 1024
N_F = 500
r_citeseer = Gen_data(cite_seer, N_F, cite_seer_seed)

In [0]:
#pubmed
pub_med_seed = 1024
N_F = 500
r_pubmed = Gen_data(pub_med, N_F, pub_med_seed)

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

## D_inv_A

In [0]:
from D_inv_A import D_inv_AConv

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = D_inv_AConv(
            num_features, num_classes, K=20, cached=True, improve= True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [99]:
das = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #r_cora_da
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-3)
  #pubmed_da
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_da
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  das.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7240
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7240
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7370
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7260
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7380
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7110
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7180
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7200
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7260
Net(
  (conv1): D_inv_AConv(1000, 3, K=20)
)

Test Accuracy 0.7230


In [100]:
np.round(das,3).tolist()

[0.724, 0.724, 0.737, 0.726, 0.738, 0.711, 0.718, 0.72, 0.726, 0.723]

In [101]:
np.mean(das)

0.7247

In [0]:
# Cora
cora_seed = 1024
N_F = 1000
r_cora = Gen_data(cora, N_F, cora_seed)

In [0]:
#Cite_Seer
cite_seer_seed = 1024
N_F = 1000
r_citeseer = Gen_data(cite_seer, N_F, cite_seer_seed)

In [0]:
#pubmed
pub_med_seed = 1024
N_F = 1000
r_pubmed = Gen_data(pub_med, N_F, pub_med_seed)

## D_A_D

In [0]:
from D_A_D import P_GCNConv

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = P_GCNConv(
            num_features, num_classes, K=20, cached=True, improve=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [87]:
dads = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #r_cora_dad
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-3)
  #pubmed_dad
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #r_cite_seer_dad
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  dads.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): P_GCNConv(1500, 3, K=20)
)

Test Accuracy 0.6960
Net(
  (conv1): P_GCNConv(1500, 3, K=20)
)


KeyboardInterrupt: ignored

In [0]:
0.5 5e-4 0.69
5e-2 0.68


In [77]:
np.round(dads,3).tolist()

[0.715, 0.715, 0.726, 0.713, 0.717, 0.721, 0.729, 0.713, 0.731, 0.726]

In [78]:
np.mean(dads)

0.7205999999999999

In [92]:
np.mean([0.715, 0.715, 0.726, 0.713, 0.717, 0.721, 0.729, 0.713, 0.731, 0.726])


0.7205999999999999

## P-AGNN

In [0]:
from cos_conv import P_AGNNConv

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = P_AGNNConv(
            num_features, num_classes, K=20,beta=1.0, cached=True, improved= True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [105]:
agnns = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #cora_agnn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-3)
  #pubmed_agnn
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_agnn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  agnns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7180
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7140
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7200
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7230
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7150
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7250
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7070
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7250
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7070
Net(
  (conv1): P_AGNNConv(1000, 3, K=20)
)

Test Accuracy 0.7240


In [106]:
np.round(agnns, 3).tolist()

[0.718, 0.714, 0.72, 0.723, 0.715, 0.725, 0.707, 0.725, 0.707, 0.724]

In [107]:
np.mean(agnns)

0.7178

# P-GAT

In [0]:
from p_gat import P_GATConv

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = P_GATConv(
            num_features, num_classes, k=20, improved=True, drop=False)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv(x, edge_index)
        return F.log_softmax(x, dim=1)

In [140]:
gats = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #r_cora_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-2)
  #r_pubmed_gat
  optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-1)


  #r_cite_seer_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=20)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gats.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.7410
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.7100
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.6930
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.6980
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.6910
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.7220
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.6740
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.7250
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.6760
Net(
  (conv): P_GATConv(1000, 3, k=20)
)

Test Accuracy 0.7110


In [0]:
lr=0.01, weight_decay=5e-3. 0.5790
lr=0.01, weight_decay=5e-2 0.6970
lr=0.1, weight_decay=5e-2 0.4410
lr=0.1, weight_decay=5e-1 0.7020*
lr=0.2, weight_decay=5e-1 0.7060



lr=0.1, weight_decay=5e-1 0.6990


In [0]:
#pubmed
lr=0.1, weight_decay=5e-4:  0.6210
lr=0.1, weight_decay=5e-3:  0.4240

lr=0.1, weight_decay=5e-3:
0.7320


lr=0.01, weight_decay=5e-4.   0.5430

lr=0.01, weight_decay=5e-3  0.6260 0.6070

lr=0.01, weight_decay=5e-2  0.5870



In [0]:
#citeseer
lr=0.01, weight_decay=5e-3: 0.4510
lr=0.01, weight_decay=5e-2: 0.4450
lr=0.01, weight_decay=5e-4: 0.4350

lr=0.02, weight_decay=5e-3: 0.4040

lr=0.2, weight_decay=5e-3: 0.2810

lr=0.0o1, weight_decay=5e-3:0.4120

In [0]:
#cora
# 有dropout的
lr=0.2, weight_decay=5e-1    0.2390
lr=0.2, weight_decay=5e-2   0.2650
lr=0.2, weight_decay=5e-2   0.2270
lr=0.2, weight_decay=5e-3   0.2790
lr=0.2, weight_decay=5e-4   0.3650
lr=0.2, weight_decay=5e-6   0.2800

lr=0.02, weight_decay=5e-4 0.1680
lr=0.5, weight_decay=5e-4. 0.1540




In [141]:
np.round(gats,3).tolist()

[0.741, 0.71, 0.693, 0.698, 0.691, 0.722, 0.674, 0.725, 0.676, 0.711]

In [142]:
np.mean(gats)

0.7041000000000001

In [143]:
np.mean([0.667, 0.649, 0.737, 0.722, 0.669, 0.687, 0.721, 0.663, 0.752, 0.697])

0.6964

In [139]:
np.mean([0.667, 0.649, 0.737, 0.722, 0.669, 0.687, 0.721, 0.663, 0.752, 0.697])


0.6964